# Setting

In [1]:
import os
from collections import defaultdict, Counter
from typing import List, Dict, Union
import pickle
import time

from tqdm import tqdm
import numpy as np
import pandas as pd
from annoy import AnnoyIndex
import polars as pl
import networkx as nx
import scipy.sparse
import scipy.sparse as sp
from scipy import linalg
from scipy.special import iv
from sklearn import preprocessing
from sklearn.utils.extmath import randomized_svd
import polars as pl

from scripts.metrics import map_at_k

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
INPUT_DIR = "../../input/raw/"
OUTPUT_DIR = "./candidates/"

TOP_N = 10

# hyper-parameter for proNE
EMB_DIM = 2048
N_EPOCH = 10
MU = 0
THETA = 0.5

In [3]:
def explode_and_add_seq_no(df:pl.DataFrame) -> pl.DataFrame:
    df = df.explode(["prev_items"])
    df = df.with_columns(
        df.select(pl.col("session_id").cumcount().over("session_id").alias("seq_no").cast(pl.Int64))
    )
    return df

In [4]:
def build_graph(df:pl.DataFrame):
    df = df.sort(["session_id", "seq_no"], descending=[False, False])
    df = df.with_columns(
        pl.col("item").shift().over("session_id").alias("prev_item")
    )
    df = df.filter(
        (pl.col("prev_item").is_not_null()) &
        (pl.col("prev_item") != pl.col("item"))
    )
    df = df[["item", "prev_item"]]
    # df = df.unique()
    return df

In [5]:
def convert_item_id(df:pl.DataFrame):
    unique_item_ids = sorted(list(set(df["item"].unique().to_list() + df["prev_item"].unique().to_list())))
    item_id2index = dict(zip(unique_item_ids, range(len(unique_item_ids))))
    df = df.with_columns([
        pl.col("item").map_dict(item_id2index).alias("item"),
        pl.col("prev_item").map_dict(item_id2index).alias("prev_item"),
    ])
    return df, unique_item_ids, item_id2index

In [6]:
# https://github.com/THUDM/ProNE/blob/master/proNE.py
class ProNE():
	def __init__(self, graph_file, emb_file1, emb_file2, dimension):
		self.graph = graph_file
		self.emb1 = emb_file1
		self.emb2 = emb_file2
		self.dimension = dimension

		self.G = nx.read_edgelist(self.graph, nodetype=int, create_using=nx.DiGraph())
		self.G = self.G.to_undirected()
		self.node_number = self.G.number_of_nodes()
		matrix0 = scipy.sparse.lil_matrix((self.node_number, self.node_number))

		for e in self.G.edges():
			if e[0] != e[1]:
				matrix0[e[0], e[1]] = 1
				matrix0[e[1], e[0]] = 1
		self.matrix0 = scipy.sparse.csr_matrix(matrix0)
		print(matrix0.shape)

	def get_embedding_rand(self, matrix):
		# Sparse randomized tSVD for fast embedding
		t1 = time.time()
		l = matrix.shape[0]
		smat = scipy.sparse.csc_matrix(matrix)  # convert to sparse CSC format
		print('svd sparse', smat.data.shape[0] * 1.0 / l ** 2)
		U, Sigma, VT = randomized_svd(smat, n_components=self.dimension, n_iter=5, random_state=None)
		U = U * np.sqrt(Sigma)
		U = preprocessing.normalize(U, "l2")
		print('sparsesvd time', time.time() - t1)
		return U

	def get_embedding_dense(self, matrix, dimension):
		# get dense embedding via SVD
		t1 = time.time()
		U, s, Vh = linalg.svd(matrix, full_matrices=False, check_finite=False, overwrite_a=True)
		U = np.array(U)
		U = U[:, :dimension]
		s = s[:dimension]
		s = np.sqrt(s)
		U = U * s
		U = preprocessing.normalize(U, "l2")
		print('densesvd time', time.time() - t1)
		return U

	def pre_factorization(self, tran, mask):
		# Network Embedding as Sparse Matrix Factorization
		t1 = time.time()
		l1 = 0.75
		C1 = preprocessing.normalize(tran, "l1")
		neg = np.array(C1.sum(axis=0))[0] ** l1

		neg = neg / neg.sum()

		neg = scipy.sparse.diags(neg, format="csr")
		neg = mask.dot(neg)
		print("neg", time.time() - t1)

		C1.data[C1.data <= 0] = 1
		neg.data[neg.data <= 0] = 1

		C1.data = np.log(C1.data)
		neg.data = np.log(neg.data)

		C1 -= neg
		F = C1
		features_matrix = self.get_embedding_rand(F)
		return features_matrix

	def chebyshev_gaussian(self, A, a, order=10, mu=0.5, s=0.5):
		# NE Enhancement via Spectral Propagation
		print('Chebyshev Series -----------------')
		t1 = time.time()

		if order == 1:
			return a

		A = sp.eye(self.node_number) + A
		DA = preprocessing.normalize(A, norm='l1')
		L = sp.eye(self.node_number) - DA

		M = L - mu * sp.eye(self.node_number)

		Lx0 = a
		Lx1 = M.dot(a)
		Lx1 = 0.5 * M.dot(Lx1) - a

		conv = iv(0, s) * Lx0
		conv -= 2 * iv(1, s) * Lx1
		for i in range(2, order):
			Lx2 = M.dot(Lx1)
			Lx2 = (M.dot(Lx2) - 2 * Lx1) - Lx0
			#         Lx2 = 2*L.dot(Lx1) - Lx0
			if i % 2 == 0:
				conv += 2 * iv(i, s) * Lx2
			else:
				conv -= 2 * iv(i, s) * Lx2
			Lx0 = Lx1
			Lx1 = Lx2
			del Lx2
			print('Bessell time', i, time.time() - t1)
		mm = A.dot(a - conv)
		emb = self.get_embedding_dense(mm, self.dimension)
		return emb
     

In [7]:
# https://github.com/THUDM/ProNE/blob/master/proNE.py
def save_embedding(emb_file, features):
	# save node embedding into emb_file with word2vec format
	f_emb = open(emb_file, 'w')
	f_emb.write(str(len(features)) + " " + str(features.shape[1]) + "\n")
	for i in range(len(features)):
		s = str(i) + " " + " ".join(str(f) for f in features[i].tolist())
		f_emb.write(s + "\n")
	f_emb.close()

In [8]:
def generate_annoy_index(matrix):
    index = AnnoyIndex(EMB_DIM, 'angular')

    for idx,idx_embedding in enumerate(matrix):
        index.add_item(idx, idx_embedding)
        
    index.build(50)
    
    return index

In [9]:
def make_nns_matrix(annoy_index, item_ids, item_id2index, k=100):
    aid_xs = []
    aid_ys = []
    dists = []

    for item_id in tqdm(item_ids):
        item_index = item_id2index[item_id]
        nns = annoy_index.get_nns_by_item(item_index, k+1, include_distances=True)
        aid_y = [item_ids[idx] for idx in list(nns[0][1:])]
        dist = list(nns[1][1:])
        aid_xs.extend([item_id] * k)
        aid_ys.extend(aid_y)
        dists.extend(dist)
    df = pl.DataFrame({"yad_no": aid_xs, 'candidate_yad_no': aid_ys, 'prone_distance': dists})

    # rankを計算
    df = df.with_columns(
        pl.col("prone_distance").rank(descending=False).over("yad_no").alias("prone_rank")
    ).drop("prone_distance")

    return df

# For local train/eval

In [10]:
train_log = pl.read_csv(os.path.join(INPUT_DIR, "train_log.csv"))

In [11]:
train_log = train_log.rename({"yad_no":"item"})

In [12]:
# build graph
graph_df = build_graph(train_log)
graph_df, item_ids, item_id2index = convert_item_id(graph_df)
graph_df.write_csv("features/edge_list_for_train_or_eval.txt", has_header=False, separator=" ")
item_ids_name = "graph_item_id2index_for_train_or_eval.pickle"
with open("features/" + item_ids_name, "wb") as f:
    pickle.dump(item_id2index, f)

# generate graph embedding by proNE 
model = ProNE(
    "features/edge_list_for_train_or_eval.txt",
    "",
    "",
    EMB_DIM,
)
features_matrix = model.pre_factorization(model.matrix0, model.matrix0)
embeddings_matrix = model.chebyshev_gaussian(model.matrix0, features_matrix, N_EPOCH, MU, THETA)
np.save("features/graph_embedding_for_train_or_eval.npy", embeddings_matrix)

# generate_candidates
annoy_index = generate_annoy_index(embeddings_matrix)
candidate = make_nns_matrix(annoy_index, item_ids, item_id2index, k=TOP_N)
candidate.write_parquet("candidates/prone_for_train_or_eval.parquet")

(11129, 11129)
neg 0.0034699440002441406
svd sparse 0.0007288705475316019
sparsesvd time 19.143075704574585
Chebyshev Series -----------------
Bessell time 2 1.537233829498291
Bessell time 3 2.3204243183135986
Bessell time 4 3.1019134521484375
Bessell time 5 3.8930490016937256
Bessell time 6 4.683181047439575
Bessell time 7 5.452115058898926
Bessell time 8 6.215810060501099
Bessell time 9 7.059511184692383
densesvd time 5.261455774307251


100% 11129/11129 [00:14<00:00, 751.87it/s]


In [13]:
candidate.head()

yad_no,candidate_yad_no,prone_rank
i64,i64,f64
2,3860,1.5
2,13783,1.5
2,12162,3.0
2,299,4.0
2,3847,5.0


# MAP@k=10

In [25]:
train_log = pl.read_csv(os.path.join(INPUT_DIR, "train_log.csv"))
train_label = pl.read_csv(os.path.join(INPUT_DIR, "train_label.csv")).rename({"yad_no":"label_yad_no"})

In [26]:
last_items = train_log.group_by("session_id").last()

In [27]:
co_visit_matrix = pl.read_parquet(os.path.join(OUTPUT_DIR, "prone_for_train_or_eval.parquet"))

In [28]:
prediction = last_items \
    .join(co_visit_matrix, on="yad_no", how="left") \
    .join(train_label, on="session_id", how="left") \
    .sort(["session_id", "prone_rank"], descending=[False, False]) \
    .with_columns((pl.col("candidate_yad_no") == pl.col("label_yad_no")).cast(pl.Int8).alias("user_relevance")) \
    .fill_null(0)

In [29]:
user_relevances = prediction.group_by("session_id", maintain_order=True).all()["user_relevance"].to_list()

In [30]:
map_at_k(user_relevances, 10)

0.16303999554870707

# For test

In [20]:
train_log = pl.read_csv(os.path.join(INPUT_DIR, "train_log.csv"))
train_label = pl.read_csv(os.path.join(INPUT_DIR, "train_label.csv"))
test_log = pl.read_csv(os.path.join(INPUT_DIR, "test_log.csv"))

In [21]:
# trainのlabelをlogにappendする

prev_items_list = (
    train_log
    .sort(["session_id", "seq_no"])
    .group_by("session_id", maintain_order=True)
    .agg(pl.col("yad_no"))
)["yad_no"].to_list()

next_item_list = (
    train_label
    .sort("session_id")
)["yad_no"].to_list()

prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)

train_log = train_label.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

train_log = explode_and_add_seq_no(train_log) \
    .drop("yad_no") \
    .rename({"prev_items" : "yad_no"}) \
    [["session_id", "seq_no", "yad_no"]] # カラム並び替え

In [22]:
log = pl.concat([train_log, test_log], how="vertical")

In [23]:
log = log.rename({"yad_no":"item"})

In [24]:
# build graph
graph_df = build_graph(log)
graph_df, item_ids, item_id2index = convert_item_id(graph_df)
graph_df.write_csv("features/edge_list_for_test.txt", has_header=False, separator=" ")
item_ids_name = "graph_item_id2index_for_test.pickle"
with open("features/" + item_ids_name, "wb") as f:
    pickle.dump(item_id2index, f)

# generate graph embedding by proNE 
model = ProNE(
    "features/edge_list_for_test.txt",
    "",
    "",
    EMB_DIM,
)
features_matrix = model.pre_factorization(model.matrix0, model.matrix0)
embeddings_matrix = model.chebyshev_gaussian(model.matrix0, features_matrix, N_EPOCH, MU, THETA)
np.save("features/graph_embedding_for_test.npy", embeddings_matrix)

# generate_candidates
annoy_index = generate_annoy_index(embeddings_matrix)
candidate = make_nns_matrix(annoy_index, item_ids, item_id2index, k=TOP_N)
candidate.write_parquet("candidates/prone_for_test.parquet")

(13806, 13806)
neg 0.005507707595825195
svd sparse 0.0013272010487664699
sparsesvd time 30.785429000854492
Chebyshev Series -----------------
Bessell time 2 2.8797013759613037
Bessell time 3 4.341245651245117
Bessell time 4 5.8141913414001465
Bessell time 5 7.27593994140625
Bessell time 6 8.754674196243286
Bessell time 7 10.24494743347168
Bessell time 8 11.697623014450073
Bessell time 9 13.153645038604736
densesvd time 6.757952928543091


100% 13806/13806 [00:22<00:00, 613.60it/s]
